# Stage 2: Filter by Metadata & Process Points

Given the geojsons created in step 1, process these into new geojson files with point data.

(This process is moved over here because node allows us to use turf)

In [40]:
var turf = require('turf')
var fs = require('fs')
var _ = require('lodash')

undefined

In [41]:
config_file = 'hajj.config'

'hajj.config'

In [42]:
var config = JSON.parse(fs.readFileSync(config_file))
config

{ tweets: '/data/hajj/gnip_tweets_full.jsonl',
  web_root: '/data/www/tweetsonamap/hajj',
  start_date: '2016-09-07' }

In [43]:
var fc = JSON.parse(fs.readFileSync(config.web_root + "/image-tweets-by-polygon.geojson"))
fc.features.length;

Error: ENOENT: no such file or directory, open '/data/www/tweetsonamap/hajj/image-tweets-by-polygon.geojson'

In [25]:
var newFeatures = []
var count       = 0
var zeroes      = []
var zero_feats  = 0;
var tweets      = 0;
var dismissed   = 0;
var dismissedTweets = 0;

fc.features.forEach(function(feat){
    var area = Math.floor(turf.area(feat.geometry)/1000000) //Area is in square kilometers
    
    if (area < 9999999){ //Empirically chosen as a limit because otherwise the areas don't make sense :) 
        if (area==0){
            //We have a point geometry
            zeroes.push(feat);
            zero_feats += feat.properties.tweets.length
        }else{
            count++;
            feat.properties.area = Math.floor(area)
            newFeatures.push(feat)
            tweets+= feat.properties.tweets.length            
        }
    }else{
        dismissed++;
        dismissedTweets += feat.properties.tweets.length
    }
})
console.log("Features kept: " + count +", tweets: " + tweets + ", dismissed: " + dismissed + " with " + dismissedTweets + " tweets")
console.log("Zeroes counted " + zeroes.length +", tweets: " + zero_feats);
null

Features kept: 3563, tweets: 22582, dismissed: 4 with 78 tweets
Zeroes counted 958, tweets: 1129


null

Write these features to disk

In [27]:
var newFC = {type: "FeatureCollection",features: _.sortBy(newFeatures.concat(zeroes),function(f){return -f.properties.area})}
fs.writeFileSync(config.web_root + '/polygon-features.geojson', JSON.stringify(newFC))

"Wrote polygon geojson with " + newFC.features.length + " polygons."

'Wrote polygon geojson with 4521 polygons.'

## Create point geometries

In [28]:
newFC.features.forEach(function(feat){
    var center = turf.center(feat.geometry)
    feat.properties.poly = JSON.stringify(feat.geometry)
    feat.geometry = center.geometry
})
"Created " + newFC.features.length + " point geometries."

'Created 4521 point geometries.'

In [29]:
fs.writeFileSync(config.web_root + '/polygon-centers-with-tweets.geojson', JSON.stringify(newFC))
"Wrote point feature collection geojson without tweets."

'Wrote point feature collection geojson without tweets.'

## Create point geometries for EVERY Tweet

In [30]:
fcTmp = JSON.parse(JSON.stringify(newFC)); null

null

In [31]:
var pt_tweets = []
fcTmp.features.forEach(function(pointWithFeatures){
    pointWithFeatures.properties.tweets.forEach(function(tweet){
        tweet.area = pointWithFeatures.properties.area
        pt_tweets.push({
            type: "Feature",
            properties: tweet,
            geometry: pointWithFeatures.geometry
        })
    })
})

undefined

In [32]:
pt_tweets.length;

23711

In [33]:
fs.writeFileSync(config.web_root + '/polygon-tweets-individual-points.geojson', JSON.stringify({'type':'FeatureCollection','features':pt_tweets}))
"Wrote point feature collection (POINT PER TWEET) geojson with " + pt_tweets.length + " tweets."

'Wrote point feature collection (POINT PER TWEET) geojson with 23711 tweets.'

## Create point geometries WITHOUT tweets

In [38]:
newFC.features.forEach(function(feat){
    delete feat.properties.tweets
})
fs.writeFileSync(config.web_root + '/polygon-centers-no-tweets.geojson', JSON.stringify(newFC))
"Wrote point feature collection geojson with " + newFC.features.length + " tweets."

'Wrote point feature collection geojson with 4521 tweets.'